In [0]:
%run Pre_ini.ipynb

In [0]:
Document = namedtuple('Document',['description','author','keywords',
                                  'title','body_a','body_h1',
                                  'body_h3','body_strong',
                                  'body_rest'])

def build_dict(words):
    counter=defaultdict(int)
    for word in words:
        counter[word]+=1
    return (dict(counter),len(words))
    

def build_docs():
    IDF_lem=defaultdict(int)
    docs=[None]*(len(get_id)+1)
    docs[0]=(dict(),Document(*[0]*9))

    i=0
    with ZipFile('all_docs_lem.zip') as myzip:
        files=myzip.namelist()
        with myzip.open(files[0]) as f:
            for line in f:
                words=line.decode('utf-8').rstrip('\n').split('\t')
                doc_id=int(words[0])



                words_description=words[1].split()

                words_author=words[2].split()

                words_keywords=words[3].split()

                words_title=words[4].split()

                words_a=words[5].split()
                words_h1=words[6].split()
                words_h3=words[7].split()
                words_strong=words[8].split()
                words_rest=words[9].split()


                all_words=set()
                feature=defaultdict(lambda:[0]*9)
                lens=[0]*9

                for ind,words in enumerate(
                    [words_description,
                    words_author,
                    words_keywords,
                    words_title,
                    words_a,
                    words_h1,
                    words_h3,
                    words_strong,
                    words_rest
                     ]):
                    for word in words:
                        feature[word][ind]+=1
                        all_words.add(word)
                    lens[ind]+=len(words)


                for word in all_words:
                    IDF_lem[word]+=1

                opt_feature=dict()
                for key in feature:
                    opt_feature[key]=Document(*feature[key])

                docs[doc_id]=(opt_feature,Document(*lens))

                i+=1
                if i%500==0:
                    print(i)
                
    return (docs,IDF_lem)


In [0]:
docs,IDF_lem=build_docs()

In [0]:
def RIDF(word,N,R,k,l):
    n=IDF_lem[word]
    if R:
        r=r_counter[word]
        ans=log(1+(r+k)*(N-n-R+r+k)/(n-r+k)/(R-r+k))
        if ans<=0:
            print(word,N,n,R,r)
            return 0
        else:
            return ans
    else:
        return log(l+(N-n)/n)

def build_avl(v):
    avl=0
    for doc in docs[1:]:
        avl+=sum(map(lambda x,y:x*y,doc[1],v))
    return avl/len(get_id)

def BM25F(query,doc_id,v,avl,R,k1,b,k,l,w_query):
    score=0
    doc=docs[doc_id]
    dl=sum(map(lambda x,y:x*y,doc[1],v))
    for word in query:
        if word in doc[0]:
            f=sum(map(lambda x,y:x*y,doc[0][word],v))
            score+=w_query[word]*(RIDF(word,len(get_id),R,k,l)*
                    (k1+1)*f/(k1*(1-b+b*dl/avl)+f))


    return score


In [0]:
candidates,_=generate_candidates()

In [0]:
Document(5,11,3,7,  1,1,1,1,1) #0.71

Document(description=5, author=11, keywords=3, title=7, body_a=1, body_h1=1, body_h3=1, body_strong=1, body_rest=1)

In [0]:
'''R=10
relevance=defaultdict(list)
with open('best.txt','r') as f:
    f.readline()
    for _ in range(399):
        for i in range(R):
            query_id,doc_id=list(map(int,f.readline().strip().split(',')))
            relevance[query_id].append(doc_id)
        for i in range(10-R):
            f.readline()
relevance[1]'''

In [50]:
%run Rus_word2vec


Loading the model...


In [0]:
process(process_pipeline, text="Это какой-то текст. ")

In [0]:
def extend(query_id,top_n):
    true_query_pos=process(process_pipeline, text=get_query[query_id])
    true_query=set(map(lambda x: x.split('_')[0],true_query_pos))
    ans=set()
    for word in true_query_pos:
        if word in model_word.vocab:
            for ext_word in model_word.most_similar(positive=[word])[:top_n]:
                if ext_word[1]<0.6:
                    continue
                ext_word=''.join(process_string(ext_word[0].split('_')[0]).split())
                ext_word=add_cash(ext_word)
                if ext_word not in true_query:
                    ans.add(ext_word)
    return ans


In [125]:
v=Document(4,12,8,10,  1,7,4,1,1) 

w_syn=0.2
top_syn=3

R=10
m=7
w_pseudo=0.35

k1=1.2
b=0.75

k=0.5
l=2


avl=build_avl(v)
best_n=10

query_ext=dict()
with open('queries.fixed.txt') as f:
    with open('BM25.txt','w') as out:         
        for i,line in enumerate(f):
            query_id,raw_query=line.split('\t')
            query=string_lem(raw_query.strip()).split()
            #true_query=set(query)
            w_query={word:1 for word in query}
            syn_query=extend(int(query_id),top_syn)
            for word in syn_query:
                w_query[word]=w_syn
                query.append(word)
            

            heap=[(-1,0)]*R

            for cand in candidates[int(query_id)]:
                heapq.heappushpop(heap,(BM25F(query,cand,v,avl,0,k1,b,k,l,w_query),cand))

            scores=sorted(heap,key=lambda x:-x[0])
            r_counter=defaultdict(int)

            for _,doc_id in scores:
                for word in docs[doc_id][0]:
                    r_counter[word]+=1

            '''r_counter=defaultdict(int)
            for doc_id in relevance[int(query_id)]:
                for word in docs[doc_id][0]:
                    r_counter[word]+=1'''

            heap=[(-1,'')]*50
            for word in r_counter:
                if r_counter[word]>1:
                    heapq.heappushpop(heap,(log(r_counter[word])*RIDF(word,len(get_id),R,k,l),word))


            
            scores=sorted(heap,key=lambda x:-x[0])
            query_ext[int(query_id)]=scores
            scores=scores[:m]
            #print('before=',query)
            
            
            for score,word in scores:
                if word not in query:
                    query.append(word)
                    w_query[word]=w_pseudo

            
            #print('after=',query)
        
            out.write(query_id+'\t')
            for cand in candidates[int(query_id)]:
                score,doc_id=(BM25F(query,cand,v,avl,R,k1,b,k,l,w_query),cand)
                out.write(str(score)+','+str(doc_id))
                out.write(' ')
            out.write('\n')

            
            if i%100==0:
                print(i)

0
100
200
300


In [0]:
with open('super_query_ext.pickle','wb') as out:
    pickle.dump(query_ext,out)